In [ ]:
#import nbpresent
#nbpresent.__version__


In [ ]:
import time

import bokeh.plotting
bokeh.plotting.output_notebook()
now=time.gmtime()
print("This notebook was last updated: "+time.asctime(now)+" UTC")

Let's compare catalogs

In [ ]:
import astropy.io.votable as votab
import astropy.io.ascii 
import astropy.units as u

import numpy as np

#download latest target lists:

latest_RV_url = "https://raw.githubusercontent.com/dsavransky/EXOSIMS/master/EXOSIMS/PlanetPopulation/RVplanets_ipac_2016-05-15.votable"
exoarch_confirmed_url = "http://exoplanetarchive.ipac.caltech.edu/cgi-bin/nstedAPI/nph-nstedAPI?table=exoplanets&format=votable&select=*"
exoplanets_eu_url = "http://www.exoplanet.eu/catalog/votable/" 
EXOSIM_stars = "https://raw.githubusercontent.com/dsavransky/EXOSIMS/master/EXOSIMS/StarCatalog/mission_exocat.votable"
oec_latest_tab = "https://raw.githubusercontent.com/douglase/oec_tables/master/comma_separated/open_exoplanet_catalogue.txt"
#right now some BD objects in the main catalog have improperly formatted names, this is fixed in the branch above.

#my fork adds it and a pull request has been submitted: https://github.com/OpenExoplanetCatalogue/oec_tables/pull/3



#(right now this misses a couple of planets that are on multiple OEC lists and are confirmed and where there confirmed status is not the first entry. 
#FIX! 

In [ ]:
#parse
exocat = votab.parse(EXOSIM_stars,datatype_mapping={"char": "unicodeChar"}).get_first_table().to_table()
#knownRV = votab.parse("../EXOSIMS/EXOSIMS/PlanetPopulation/RVplanets_ipac_032216.votable").get_first_table().to_table()
knownRV = votab.parse(latest_RV_url,datatype_mapping={"char": "unicodeChar"}).get_first_table().to_table()
#exoplanets_dot_eu = votab.parse(exoplanets_eu).get_first_table().to_table()
#create 

exoarch_confirmed_file=astropy.utils.data.download_file(exoarch_confirmed_url, 
                                                        cache=False,
                                                        show_progress=True,
                                                        timeout=10) #regular call was timing out
exoarchConfirmed = votab.parse(exoarch_confirmed_file,datatype_mapping={"char": "unicodeChar"}).get_first_table().to_table()
last_download_time=time.asctime(now)

In [ ]:
if exoarchConfirmed["pl_orbper"].unit == "days":
    print("days not a recognized unit, replacing with day")
    exoarchConfirmed["pl_orbper"].unit=u.day #fix units
if knownRV["pl_orbper"].unit == "days":
    print("days not a recognized unit, replacing with day")
    knownRV["pl_orbper"].unit=u.day #fix units


In [ ]:
'''exoplanets_eu_file = astropy.utils.data.download_file(exoplanets_eu_url, 
                                                        cache=False,
                                                        show_progress=True,
                                                        timeout=60) 
exoplanets_dot_eu = votab.parse(exoplanets_eu_file)#.get_first_table().to_table() #returns E05: Invalid boolean value 'Yes'
'''

In [ ]:
from astroquery.vizier import Vizier
import astropy.units as u
def get_hip(target_name):
    viz=Vizier.query_object(target_name, catalog="I/311/hip2") 
    if len(viz.values()) ==0:
        return target_name
    return "HIP  "+str(viz['I/311/hip2']["HIP"][0])
get_hip("eps eri")
def get_2MASS(target_name):
    viz=Vizier.query_object(target_name, catalog="II/246/out",radius=5*u.arcsecond) 
    if len(viz.values()) ==0:
        return target_name
    return "2MASS "+str(viz["II/246/out"]["_2MASS"][0].decode('utf-8') )
get_hip("eps eri"),get_2MASS("eps eri")

In [ ]:
#Define open exoplanet catalog column names:
oec_cols=["name",#  1: Primary identifier of planet
          "binary",#  2: Binary flag [0=no known stellar binary companion; 1=P-type binary (circumbinary); 2=S-type binary; 3=orphan planet (no star)]
          "pl_massj",##  3: Planetary mass [Jupiter masses]
          "pl_radiusj",##  4: Radius [Jupiter radii]
          "pl_orbper",#  5: Period [days]
          "pl_orbsmax",#  6: Semi-major axis [Astronomical Units]
          "pl_orbeccen",#  7: Eccentricity
          "pl_orblper",#  8: Periastron [degree]
          "longitude", ##  9: Longitude [degree] https://github.com/OpenExoplanetCatalogue/open_exoplanet_catalogue/issues/292
          "ascendingnode",#"10: Ascending node [degree]",
          "pl_orbincl",#"# 11: Inclination [degree]
          "pl_eqt",# 12: Surface or equilibrium temperature [K]
          "pl_age",# 13: Age [Gyr]
          "pl_discmethod",#,14: Discovery method
          "pl_disc",# 15:" Discovery year [yyyy]
          "rowupdate",# 16: Last updated [yy/mm/dd]
          "ra_str",# 17: Right ascension [hh mm ss]
          "dec_str",# 18: Declination [+/-dd mm ss]
          "st_dist",# 19: Distance from Sun [parsec]
          "st_mass",# 20: Host star mass [Solar masses]
          "st_rad",# 21: Host star radius [Solar radii]
          "st_metfe",# 22: Host star metallicity [log relative to solar]
          "st_teff",# 23: Host star temperature [K],"24: Host star age [Gyr]"]
          "st_age",# 24: Host star age [Gyr]
          "list",#detection status list
]
          
oec=astropy.io.ascii.read(oec_latest_tab,data_start=31,names=oec_cols,delimiter=",",guess=False)
oec_all = oec.copy()
oec=oec[oec["list"]=="Confirmed planets"] #throw away planets that aren't marked confirmed
oec
oec["pl_orbper"].unit=u.day
oec["st_dist"].unit=u.parsec


In [ ]:
import pandas as pd
curated = pd.read_excel("../../projects/wfirst/WFIRST_prime_RV_targets_082016.xlsx",
                        header=0,skiprows=1, encoding="utf-8-sig",
                        index_col=None)
#curated["DIST"]

In [ ]:
exoc_pd = exocat.to_pandas()
exoarc_pd = exoarchConfirmed.to_pandas()
knownRV_pd=knownRV.to_pandas()

oec_pd=oec.to_pandas()
oec_all_pd=oec_all.to_pandas()


In [ ]:
import cat_proc
from importlib import reload
reload(cat_proc)
#test that the Simbad query works (need internet...):
assert cat_proc.get_names("Epsilon Eridani",catalog="HIP") == 'HIP  16537'
assert get_hip("epsilon eridani") == 'HIP  16537'



curated["hip_name"]=curated["NAME"]
for i in curated["hip_name"].index:
    name=curated.loc[i,"NAME"]
    curated.loc[i,"hip_name"]=(get_hip(name[:-2])+name[-2:]).strip("\n").strip()
        

oec_nearby_pd=oec_all_pd[(oec_all_pd["st_dist"]<50) & (oec_all_pd["pl_orbsmax"]>1)]
oec_nearby_pd["hip_name"]=oec_nearby_pd["name"]

for i in oec_nearby_pd["hip_name"].index:
    name = oec_nearby_pd.loc[i,"name"]

    oec_nearby_pd.loc[i,"hip_name"]=(get_hip(name[:-2])+name[-2:]).strip("\n").strip()
        

In [ ]:
oec_nearby_pd_HP=oec_nearby_pd.set_index("hip_name",inplace=False)
curated_HP=curated.set_index("hip_name",inplace=False)
curated_HP

In [ ]:
result = pd.merge(curated_HP, oec_nearby_pd_HP,how='left',
                  left_index=True,right_index=True)#,join='outer',axis=0)
result["%Mj"]=np.abs ((result["MSINI"]-result["pl_massj"])/result["MSINI"]*100)
result["%AU"]=np.abs((result["A"]-result["pl_orbsmax"])/result["A"]*100)
result.shape

updates=result.copy()
updates[["NAME","MSINI","pl_massj","%Mj","A","pl_orbsmax","%AU","Notes"]].sort_values("%Mj")

updates["ADSbibcode"]=" "
#updates["Notes"]=" "


#update parameters
#HD 217107 c: 
"""
Table 3
List of Keplerian Orbital Parameters
Name P              Tp                e              ω          K             Msini                a 
b 7.126846 ± 0.000013 14395.789 ± 0.025 0.1283 ± 0.0027 24.0 ± 1.3 140.30 ± 0.40 1.4135 ± 0.0042 0.07505 ± 0.00097
c 5189 ± 21               10770 ± 16    0.3848 ± 0.0086 206.3 ± 1.7 53.41 ± 0.75  4.513 ± 0.072    6.074 ± 0.080"""
updates.loc['HIP  113421 c',"ADSbibcode"] = "2015ApJ...800...22F" 
updates.loc['HIP  113421 c',"A"] = 6.074 #HD 217107 c
updates.loc['HIP  113421 c',"PER"] = 5189 #HD 217107 c
updates.loc['HIP  113421 c',"ECC"] = 0.3848 #HD 217107 c
updates.loc['HIP  113421 c',"OM"] = 206.3 #HD 217107 c
updates.loc['HIP  113421 c',"T0"] = 10770+2440000 #HD 217107 c



#eps eri:
"""
Eridani Including Seven Data Sets
Parameter CCF
P [days] 2651 (36)
K [m s−1] 11.8 (1.1)
M0 [deg] 09 (12)
ea 0.40 (11)
ω[deg] 141.4 (9.8)
M sin i [Mjup] 0.645 (58)
a [AU] 3.51

"""
updates.loc['HIP  16537 b',"ADSbibcode"] = "2012ApJS..200...15A" 
updates.loc['HIP  16537 b',"A"] = 3.51 #
updates.loc['HIP  16537 b',"MSINI"] = 0.645
updates.loc['HIP  16537 b',"PER"] = 2651 #
updates.loc['HIP  16537 b',"ECC"] = 0.40 #
updates.loc['HIP  16537 b',"OM"] = 141.4 #
# T0 (initial mean anamoly) not provided


#gamma ceph b:
# oec, exoplanets.eu and exoplanet archive all cite http://adsabs.harvard.edu/abs/2011AIPC.1331...88E
# the exoplanets.org value is from http://adsabs.harvard.edu/abs/2003ApJ...599.1383H
'''
TABLE 1. New orbital and planetary parameters
for g Cep Ab
parameter value error units
Period 903.3 1.5 [days]
K 31.1 0.97 [ms−1]
e 0.049 0.034
Tper 2453227 87 [days]
w 94.6 34.6 [deg]
m sini 1.85 0.16 [MJup]
a 2.05 0.06 [AU]
'''
updates.loc['HIP  116727 b',"ADSbibcode"] = "2011AIPC.1331...88E" 
updates.loc['HIP  116727 b',"A"] = 2.05 #HD 217107 c
updates.loc['HIP  116727 b',"MSINI"] = 1.85 #HD 217107 c
updates.loc['HIP  116727 b',"PER"] = 903.3#
updates.loc['HIP  116727 b',"ECC"] = 0.049 #
updates.loc['HIP  116727 b',"OM"] = 94.6 #
updates.loc['HIP  116727 b',"T0"] = 2453227
updates["%AU"]=np.abs((updates["A"]-updates["pl_orbsmax"])/updates["A"]*100)
#updates["Separation"]=updates["A"]/updates["DIST"]


updates[["NAME","MSINI","pl_massj","%Mj","A","pl_orbsmax","%AU","Separation","ADSbibcode","Notes"]].sort_values("%Mj")

#remove the error columns
cleaned = updates.copy()
cleaned.pop("%AU")
cleaned.pop("%Mj")


from astropy.time import Time
from astropy.table import Table
import astropy.units as u
latest_params_fname = "LatestRVmeasurements_enh.csv"

#drop the derived parameters:
measured_parameters =  cleaned[curated.columns[0:10]]
measured_parameters["Notes"]=cleaned["Notes"]
measured_parameters["hip_name"]=measured_parameters.index


#switch to VOTable to preserve units
votab_measured=Table.from_pandas(measured_parameters)
votab_measured["MSINI"].unit = u.jupiterMass
votab_measured["PER"].unit = u.day
votab_measured["A"].unit = u.AU
votab_measured["OM"].unit = u.degree
votab_measured["T0"].unit = u.day
votab_measured["V"].unit = u.mag
votab_measured["DIST"].unit = u.parsec

votab_measured.write(latest_params_fname,format="ascii.ecsv",delimiter=',')
#read table back to make sure worked:
Table.read(latest_params_fname,format="ascii.ecsv")


In [ ]:
cleaned["Notes"]

In [ ]:
def scatter_with_labels(p,x,y,names,**kwargs):
    #data source for mouseover
    if isinstance(names[0],bytes):
        names= [j.decode('utf-8') for j in names]
    source = bokeh.models.ColumnDataSource(data=dict(
        x=x,
        y=y,
        name=names, #convert bytes to strings
    ))

    p.scatter(x,y,line_color=None,source=source,**kwargs)
tooltips=[("index", "$index"),
          ("(x,y)", "(@x, @y)"),
          ("name", "@name")]

In [ ]:

TOOLS="hover,crosshair,pan,wheel_zoom,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select"

p = bokeh.plotting.figure(tools=TOOLS, x_range=( 10000,0))

p.scatter(exocat['st_teff'], (exocat['st_mass']), radius=50,
          fill_color='gray', fill_alpha=0.5,legend="ExoCat",
          line_color=None)

p.scatter(knownRV['st_teff'], (knownRV["st_mass"]), radius=100,
fill_color="orange", fill_alpha=0.2,legend="Known RV",
          line_color=None)
p.xaxis.axis_label = "T [K]"
p.yaxis.axis_label = "Mass [M_sun]"
bokeh.plotting.show(p)


In [ ]:

hover = bokeh.models.HoverTool(tooltips=tooltips)
tools=[hover]+["crosshair,pan,wheel_zoom,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select"]
p = bokeh.plotting.figure(tools=tools,y_axis_type='log', y_range=(1e-3,1e3) ,x_range=( 10000,0))

scatter_with_labels(p,exocat['st_teff'],exocat['st_lbol'],
                    exocat["hip_name"],
                    color="blue",legend="exocat",size=10,fill_alpha=0.1)

scatter_with_labels(p,knownRV["st_teff"],10**(knownRV["st_lum"]), 
                    knownRV["pl_name"],
                    color="orange",legend="Known RV",size=10,fill_alpha=.3)

p.title.text = "H-R Diagram"
p.xaxis.axis_label = "T [K]"
p.yaxis.axis_label = "Luminousity [L_sun]"
bokeh.plotting.show(p)


In [ ]:

hover = bokeh.models.HoverTool(tooltips=tooltips)
tools=[hover]+["crosshair,pan,wheel_zoom,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select"]
p = bokeh.plotting.figure(tools=tools,y_axis_type='log', y_range=(1,1e3) ,x_range=( 10000,0))


p.scatter(exocat['st_teff'], (exocat['st_dist']), 
                              exocat["hip_name"],
          size=5,
          fill_color='gray', fill_alpha=0.2,legend="ExoCat",
          line_color=None)

p.scatter(knownRV['st_teff'], (knownRV["st_dist"]),
                              knownRV["pl_name"], size=10,
          fill_color="blue", fill_alpha=0.2,legend="Known RV",
          line_color=None)

p.scatter(exoarchConfirmed['st_teff'], (exoarchConfirmed["st_dist"]), size=10,
fill_color="green", fill_alpha=0.1,legend="IPAC Exoplanet Archive",
          line_color=None)
p.xaxis.axis_label = "T [K]"
p.yaxis.axis_label = "distance [pc]"
bokeh.plotting.show(p)




In [ ]:

hover = bokeh.models.HoverTool(tooltips=tooltips)
tools=[hover]+["crosshair,pan,wheel_zoom,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select"]
p = bokeh.plotting.figure(tools=tools,y_axis_type='log', y_range=(1,1e3) ,x_range=( 10000,0))

scatter_with_labels(p,exocat['st_teff'], (exocat['st_dist']),
                    exocat["hip_name"],
                    radius=50,
          fill_color='gray', fill_alpha=0.2,legend="ExoCat",)
scatter_with_labels(p,knownRV['st_teff'], (knownRV["st_dist"]), knownRV["pl_name"],
                    radius=100,
                    fill_color="blue", fill_alpha=0.2,legend="Known RV",)
scatter_with_labels(p,exoarchConfirmed['st_teff'], (exoarchConfirmed["st_dist"]), exoarchConfirmed["pl_hostname"],
                    radius=100,
                    fill_color="green", fill_alpha=0.2,legend="IPAC Exoplanet Archive",)
p.xaxis.axis_label = "T [K]"
p.yaxis.axis_label = "distance [pc]"
bokeh.plotting.show(p)



In [ ]:
hover = bokeh.models.HoverTool(tooltips=tooltips)
tools=[hover]+["crosshair,pan,wheel_zoom,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select"]

p = bokeh.plotting.figure(tools=tools,x_axis_type="log",  y_range=(-5, 35),x_range=(0, 25),  )

scatter_with_labels(p,exoarchConfirmed["st_optmag"],exoarchConfirmed["st_dist"],
                    exoarchConfirmed["pl_hostname"],
                    color="green",legend="IPAC Exoplanet Archive",
                    size=12,fill_alpha=0.34)

scatter_with_labels(p,knownRV["st_optmag"],knownRV["st_dist"], 
                    knownRV["pl_hostname"],
                    size=12,fill_alpha=0.34,
                    color="blue",legend="Known RV")

#bokeh.plotting.show(p)
p.xaxis.axis_label = "Optical Mag"
p.yaxis.axis_label = "distance [pc]"

#what is optical mag? Johnson v_mag?




In [ ]:
hover = bokeh.models.HoverTool(tooltips=tooltips)


tools=[hover]+["crosshair,pan,wheel_zoom,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select"]

p = bokeh.plotting.figure(tools=tools,x_axis_type="log",  y_range=(-5, 35))#x_range=(0, 10000),  )

scatter_with_labels(p,exoarchConfirmed["pl_orbper"],exoarchConfirmed["st_dist"],
                    exoarchConfirmed["pl_hostname"],
                    color="gray",legend="IPAC Exoplanet Archive",size=6,fill_alpha=0.6)

scatter_with_labels(p,knownRV["pl_orbper"],knownRV["st_dist"], 
                    knownRV["pl_hostname"],
                    color="blue",legend="Known RV",size=10,fill_alpha=.35)

scatter_with_labels(p,oec["pl_orbper"],oec["st_dist"], 
                    oec["name"],
                    color="orange",legend="Open Exoplanet Catalog Confirmed",size=10,fill_alpha=.35)
p.xaxis.axis_label = "orbital period [d]"
p.yaxis.axis_label = "distance [pc]"



In [ ]:

def sep_as(dist,period):
    '''
    calculate the separation in arcseconds
    $p[years]^2=a[AU]^3$

    s["]=a[AU]/d[pc]

    $s["]=(p)^{2/3}/d$
    '''
    return (period.to(u.year).value**(2/3)/dist.to(u.parsec).value)
hover = bokeh.models.HoverTool(tooltips=tooltips)


tools=[hover]+["crosshair,pan,wheel_zoom,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select"]

p2 = bokeh.plotting.Figure(tools=tools,x_axis_type="log",  y_range=(-5, 45),x_range=(1e-2, 40),  )
#https://github.com/bokeh/bokeh/issues/3531

scatter_with_labels(p2,sep_as(exoarchConfirmed["st_dist"],exoarchConfirmed["pl_orbper"]),
                    exoarchConfirmed["st_dist"],
                     exoarchConfirmed["pl_hostname"] +exoarchConfirmed["pl_letter"],        
                    color="gray",legend="IPAC Exoplanet Archive",size=10,fill_alpha=0.6)

scatter_with_labels(p2,sep_as(knownRV["st_dist"],knownRV["pl_orbper"]),knownRV["st_dist"], 
                    knownRV["pl_hostname"] +knownRV["pl_letter"],
                    color="blue",legend="EXOSIMS Known RV",size=10,fill_alpha=.35)

scatter_with_labels(p2,sep_as(oec["st_dist"],oec["pl_orbper"]),oec["st_dist"], 
                    oec["name"],
                    color="orange",legend="Open Exoplanet Catalog Confirmed",size=10,fill_alpha=.35)
p2.xaxis.axis_label = "separation [arcseconds]"
p2.yaxis.axis_label = "distance [pc]"


p2.title.text = "Separation (from period, assuming circular orbits)"


#bokeh.plotting.show(p2)

In [ ]:
#g = plt.scatter(x="MSINI", y="Separation", data=curated,alpha=0.5,color="blue",s=30)
#g = plt.scatter(x="MSINI", y="Separation", data=updates,alpha=0.5,color="red",s=30)
#plt.xscale("log")
#plt.yscale("log")

bokeh.io.curdoc().clear()#https://github.com/bokeh/bokeh/issues/3943


hover = bokeh.models.HoverTool(tooltips=tooltips)


tools=[hover]+["crosshair,pan,wheel_zoom,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select"]

p3 = bokeh.plotting.Figure(tools=tools,x_axis_type="log",#y_axis_type="log",  
                           y_range=(0.01e-1, 1.5),x_range=(1e-3, 30),  )
#https://github.com/bokeh/bokeh/issues/3531
scatter_with_labels(p3,curated["MSINI"],
                    curated["Separation"], 
                     curated["NAME"],
                    size=20,fill_alpha=0.34,
                    color="blue",legend="curated",marker='square')
scatter_with_labels(p3,updates["MSINI"],
                    updates["Separation"], 
                     updates["NAME"],
                    size=20,fill_alpha=0.34,
                    color="green",legend="Updated",marker='square')
bokeh.plotting.show(p3)

In [ ]:
bokeh.io.curdoc().clear()#https://github.com/bokeh/bokeh/issues/3943

hover = bokeh.models.HoverTool(tooltips=tooltips)
tools=[hover]+["crosshair,pan,wheel_zoom,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select"]

p1 = bokeh.plotting.Figure(tools=tools,x_axis_type="log",  y_range=(-5, 75),x_range=(1e-1, 40),  )

scatter_with_labels(p1,exoarchConfirmed["pl_msinij"],exoarchConfirmed["st_dist"],
                     exoarchConfirmed["pl_hostname"] +exoarchConfirmed["pl_letter"],        
                    size=12,fill_alpha=0.6,
                    color="gray",legend="IPAC Exoplanet Archive")

scatter_with_labels(p1,knownRV["pl_msinij"],knownRV["st_dist"], 
                     knownRV["pl_hostname"] +knownRV["pl_letter"],
                    size=12,fill_alpha=0.34,
                    color="blue",legend="EXOSIMS Known RV")

scatter_with_labels(p1,curated["MSINI"],curated["DIST"], 
                     curated["NAME"],
                    size=20,fill_alpha=0.34,
                    color="red",legend="curated",marker='square')

scatter_with_labels(p1,oec["pl_massj"],oec["st_dist"], 
                    np.array(oec["name"],dtype=object)+ np.array(oec["pl_discmethod"],dtype=object),
                    color="orange",legend="Open Exoplanet Catalog Confirmed Planets",size=10,fill_alpha=.35)
p1.xaxis.axis_label = "orbital period [d]"
scatter_with_labels(p1,oec["pl_massj"],oec["st_dist"],oec["name"],
                    size=10,color='orange',fill_alpha=.3)
p1.xaxis.axis_label = "msini [Jupiter]"
p1.yaxis.axis_label = "distance [pc]"
p1.title.text = "Catalogs last queried: "+last_download_time
#bokeh.plotting.show(p1)

In [ ]:
bokeh.io.curdoc().clear()#https://github.com/bokeh/bokeh/issues/3943


hover = bokeh.models.HoverTool(tooltips=tooltips)


tools=[hover]+["crosshair,pan,wheel_zoom,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select"]

p3 = bokeh.plotting.Figure(tools=tools,x_axis_type="log",y_axis_type="log",  
                           y_range=(0.3e-1, 40),x_range=(1e-2, 40),  )
#https://github.com/bokeh/bokeh/issues/3531

scatter_with_labels(p3,exoarchConfirmed["pl_msinij"],
                    sep_as(exoarchConfirmed["st_dist"],exoarchConfirmed["pl_orbper"]),
                     exoarchConfirmed["pl_hostname"] +exoarchConfirmed["pl_letter"],        
                    color="gray",legend="IPAC Exoplanet Archive (msini)",size=10,fill_alpha=0.6)
scatter_with_labels(p3,exoarchConfirmed["pl_bmassj"],
                    sep_as(exoarchConfirmed["st_dist"],exoarchConfirmed["pl_orbper"]),
                     exoarchConfirmed["pl_hostname"] +exoarchConfirmed["pl_letter"]+ exoarchConfirmed["pl_discmethod"],        
                    color="gray",legend="IPAC Exoplanet Archive (Best Mass)",
                    size=10,fill_alpha=0.6,marker="diamond")

scatter_with_labels(p3,                    knownRV["pl_msinij"],
                    sep_as(knownRV["st_dist"],knownRV["pl_orbper"]),
                    knownRV["pl_hostname"] +knownRV["pl_letter"],
                    color="blue",legend="EXOSIMS Known RV",size=10,fill_alpha=.35)

scatter_with_labels(p3,oec["pl_massj"], 
                    sep_as(oec["st_dist"],oec["pl_orbper"]),
                    oec["name"],
                    color="orange",legend="Open Exoplanet Catalog Confirmed",size=10,fill_alpha=.35)

scatter_with_labels(p3,curated["MSINI"],
                    curated["Separation"], 
                     curated["NAME"],
                    size=20,fill_alpha=0.34,
                    color="red",legend="curated",marker='square')


p3.yaxis.axis_label = "separation [arcseconds]"
p3.xaxis.axis_label = "mass [Jupiter]"


p3.title.text = "Separation  (from period, assuming circular orbits) vs mass"


#bokeh.plotting.show(p3)

In [ ]:
grid = bokeh.layouts.gridplot([p2,p1,p3], ncols=2, plot_width=450, plot_height=600)
bokeh.plotting.output_file("pc_msini_and_sep.html", title=last_download_time)
#bokeh.plotting.show(p)

In [ ]:
bokeh.plotting.show(grid)

### 


In [ ]:

oec_nearby_pd_HP

In [ ]:
updates.shape